In [1]:
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# 데이터 전처리
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

In [29]:
import platform
platform.system()
# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정

%config InlineBackend.figure_format = 'retina'

In [13]:
df_d = pd.read_csv('data_dong.csv', encoding='utf-8')
# 자동차 등록대수 열 = 자동차 등록대수 열 / 인구수 열
df_d['자동차 등록대수'] = df_d['자동차 등록대수'] / df_d['인구수']
# 행정동명 열 삭제
df_d = df_d.drop(['행정동명'], axis=1)
# 행정구 행정동 열을 합쳐 row name으로 사용하고 두 열 삭제
df_d = df_d.set_index(['행정구', '행정동'])
df_d = df_d.drop(['Column1'], axis=1)
df_d

인구수   사업체수    종사자수   집 개수       소득    10대 비율    20대 비율   3040 비율  \
행정구  행정동                                                                        
송파구  가락1동  27670   3791   12367   9716  4973588  0.096314  0.095699  0.354355   
     가락2동  31450   3155   11544  10424  3910198  0.103529  0.130175  0.310715   
     가락본동  25617   5069   28151   8410  3521561  0.063552  0.140571  0.318070   
구로구  가리봉동  12478   1189    4248   2551  2336331  0.025325  0.146818  0.285062   
금천구  가산동   27166  25043  182376   6200  2567594  0.025068  0.268571  0.376058   
...          ...    ...     ...    ...      ...       ...       ...       ...   
용산구  효창동   10828    702    2596   3463  3523201  0.062615  0.152290  0.337828   
     후암동   16540   1367    4605   5363  3013677  0.068622  0.137485  0.320798   
동대문구 휘경1동  16168   1285    3690   4327  2722774  0.055418  0.348714  0.244805   
     휘경2동  25288   1665    5457   6802  3071586  0.077349  0.261152  0.251028   
동작구  흑석동   32366   2136   11037   9613  4224946  0.074337  0.195298  0.279398   

           65세 이상 비율  버스정류장 개수  지하철역 개수  자동차 등록대수  도로 평균 속도  
행정구  행정동                                                     
송파구  가락1동   0.142212        14        3  0.412215     16.42  
     가락2동   0.149571        21        2  0.342067     15.61  
     가락본동   0.164930        25        1  0.491314     16.24  
구로구  가리봉동   0.178875         9        0  0.218785     15.08  
금천구  가산동    0.110248       105        3  0.736582     18.46  
...              ...       ...      ...       ...       ...  
용산구  효창동    0.147950        15        0  0.299778     16.11  
     후암동    0.185248        16        0  0.242684     18.46  
동대문구 휘경1동   0.135267        13        1  0.243196     22.15  
     휘경2동   0.167550        18        0  0.251226     15.97  
동작구  흑석동    0.160353        48        1  0.281067     15.36  

[425 rows x 13 columns]

In [15]:
df_popul_company = df_d.drop(['종사자수', '집 개수'], axis=1)
df_popul_company

인구수   사업체수       소득    10대 비율    20대 비율   3040 비율  65세 이상 비율  \
행정구  행정동                                                                    
송파구  가락1동  27670   3791  4973588  0.096314  0.095699  0.354355   0.142212   
     가락2동  31450   3155  3910198  0.103529  0.130175  0.310715   0.149571   
     가락본동  25617   5069  3521561  0.063552  0.140571  0.318070   0.164930   
구로구  가리봉동  12478   1189  2336331  0.025325  0.146818  0.285062   0.178875   
금천구  가산동   27166  25043  2567594  0.025068  0.268571  0.376058   0.110248   
...          ...    ...      ...       ...       ...       ...        ...   
용산구  효창동   10828    702  3523201  0.062615  0.152290  0.337828   0.147950   
     후암동   16540   1367  3013677  0.068622  0.137485  0.320798   0.185248   
동대문구 휘경1동  16168   1285  2722774  0.055418  0.348714  0.244805   0.135267   
     휘경2동  25288   1665  3071586  0.077349  0.261152  0.251028   0.167550   
동작구  흑석동   32366   2136  4224946  0.074337  0.195298  0.279398   0.160353   

           버스정류장 개수  지하철역 개수  자동차 등록대수  도로 평균 속도  
행정구  행정동                                          
송파구  가락1동        14        3  0.412215     16.42  
     가락2동        21        2  0.342067     15.61  
     가락본동        25        1  0.491314     16.24  
구로구  가리봉동         9        0  0.218785     15.08  
금천구  가산동        105        3  0.736582     18.46  
...             ...      ...       ...       ...  
용산구  효창동         15        0  0.299778     16.11  
     후암동         16        0  0.242684     18.46  
동대문구 휘경1동        13        1  0.243196     22.15  
     휘경2동        18        0  0.251226     15.97  
동작구  흑석동         48        1  0.281067     15.36  

[425 rows x 11 columns]

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import matplotlib.pyplot as plt

def lasso_ridge_graph(df, target_column):
    X = df.drop([target_column], axis=1)
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 2021)
    X_train_final = X_train
    X_test_final = X_train
    X_train, y_train = X_train.values, y_train.values
    X_test, y_test = X_test.values, y_test.values

    # 모델 학습
    model_LR = LinearRegression(n_jobs = -1)
    model_LR.fit(X_train, y_train)

    # 계수 출력
    display(pd.DataFrame(model_LR.coef_,
        index = X_test_final.columns, columns = ['Linear regression 계수']))

    model_Lasso1 = Lasso(alpha = 0.0001, random_state = 1)
    model_Lasso2 = Lasso(alpha = 100.0, random_state = 1)
    model_Ridge1 = Ridge(alpha = 0.0001, random_state = 1)
    model_Ridge2 = Ridge(alpha = 100.0, random_state = 1)

    model_Lasso1.fit(X_train, y_train)
    model_Lasso2.fit(X_train, y_train)
    model_Ridge1.fit(X_train, y_train)
    model_Ridge2.fit(X_train, y_train)

    plt.figure(figsize = (10, 5))
    plt.plot(sorted(np.abs(model_LR.coef_))[::-1], label = 'LR', c = 'gray')
    plt.plot(sorted(np.abs(model_Lasso1.coef_))[::-1], label = 'Lasso alpha = 0.0001')
    plt.plot(sorted(np.abs(model_Lasso2.coef_))[::-1], label = 'Lasso alpha = 100')
    plt.plot(sorted(np.abs(model_Ridge1.coef_))[::-1], label = 'Ridge alpha = 0.0001')
    plt.plot(sorted(np.abs(model_Ridge2.coef_))[::-1], label = 'Ridge alpha = 100')
    plt.legend()
    plt.ylim((-0.1, 1000))
    plt.show()

In [32]:
from sklearn.linear_model import LassoCV

def find_alpha(df, target_column, alphas):
    X = df.drop([target_column], axis=1)
    y = df[target_column]
    X_train, y_train = X.values, y.values

    # LassoCV 모델 생성
    lasso_cv = LassoCV(alphas=alphas, cv=5)  # cv는 교차 검증의 폴드(fold) 수를 나타냄

    # 모델 훈련
    lasso_cv.fit(X_train, y_train)

    # 최적의 alpha 값 출력
    print("Optimal alpha:", lasso_cv.alpha_)

    return lasso_cv

In [26]:
import matplotlib.pyplot as plt

def plot_coef(columns, coef):
    coef_df = pd.DataFrame(list(zip(columns, coef)), columns=['Features', 'Coefficients'])
    coef_df = coef_df.sort_values(by='Coefficients')
    
    plt.figure(figsize=(8, 10))
    plt.barh(coef_df['Features'], coef_df['Coefficients'])
    plt.xlabel('Coefficients')
    plt.ylabel('Features')
    plt.title('Feature Coefficients')
    plt.show()

In [33]:
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt

def train_lasso(df, target_column, alpha):
    X = df.drop([target_column], axis=1)
    y = df[target_column]
    X_train, y_train = X.values, y.values

    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    lasso_pred = lasso.predict(X_train)

    plot_coef(X.columns, lasso.coef_)
    print(lasso.coef_)

    return lasso, lasso_pred

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def plot_heatmap(df):
    num_cols = df.select_dtypes(include=[np.number]).columns
    corr = df[num_cols].corr()
    mask = np.triu(np.ones_like(corr))

    plt.figure(figsize = [15,10])
    sns.heatmap(corr[np.abs(corr) > 0.5], annot = True, mask = mask, cmap = 'Blues_r', fmt = '.1g')
    plt.show()

In [35]:
import statsmodels.api as sm

def calculate_aic(df, target_column):
    X = df.drop([target_column], axis=1)
    y = df[target_column]
    X = sm.add_constant(X)  # Add a constant term like so the model would be y = Xb + c

    model = sm.OLS(y, X)
    results = model.fit()

    return results.aic

In [40]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

def calculate_vif(df, target_column):
    X = df.drop([target_column], axis=1)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns

    # Calculating VIF for each feature
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

    return vif_data